
# Nested Experiment Replication

When migrating models, there is an intent to migrate the following things from WS to UC:
 - `WS Registered Model` -> `UC Registered Model`
 - `Notebook Experiments` -> `Workspace Experiments` - [Optional]


Nesting can come about from either from intentional logging in nested experiment or through the use of early documented hyperopt-mlflow integrations. This replication is to provide an experiment + registered model that has this complexity for migration code testing.

In [0]:
#%pip install --upgrade "mlflow-skinny[databricks]==2.20.0"
%pip install --upgrade "mlflow[databricks]==2.20.0"
%pip install --upgrade "scikit-learn==1.6.1"
%pip install --upgrade "hyperopt==0.2.7"
dbutils.library.restartPython()


This replication will do the following tasks:
 * Create a WS Experiment
 * Log four models, then register as Workspace Registered Models
 * Delete two of the four experiment runs

In [0]:
import mlflow
from mlflow.tracking import MlflowClient

# Explicitly set the model registry to the WS model registry, ie NOT "datarbricks-uc"
client = MlflowClient(tracking_uri="databricks",
                      registry_uri="databricks")

model_name="nested"
experiment_name = f"/Workspace/experiments/uc_ml_migrate/{model_name}"
#experiment_name = dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()

try:
    experiment_id = client.create_experiment(name=experiment_name)
    experiment = client.get_experiment(experiment_id=experiment_id)
    for i in range(3):
            client.set_experiment_tag(experiment_id, f"exp_tag{i}", i)
except:
    experiment = client.search_experiments(filter_string=f'name="{experiment_name[10:]}"')[0]
    experiment_id = experiment.experiment_id


### Model Prep

In model prep, we are going to create all of the artifacts that will go into logging models including the mode important sk_model `lr`, and `input_example` which will be needed when we log our model.

In [0]:
# This is code that is representative of complete record whe doing hyperparameter tuning. 
# In this convention there is a parent run that collects hyper parameter tuning runs - the best model is logged in the parent experiment run, and
#  it is the child experiment run that is registered in the Model Registry.

import mlflow
import mlflow.sklearn
import numpy as np
import pandas as pd
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

# Enable MLflow autologging (Optional but useful)
mlflow.sklearn.autolog()

# Load sample dataset
from sklearn.datasets import make_regression

X, y = make_regression(n_samples=500, n_features=10, noise=0.1, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

parent_run_id=""

# Define objective function for Hyperopt
def objective(params):
    alpha = params["alpha"]

    # Enable MLflow logging within Hyperopt
    with mlflow.start_run(nested=True, experiment_id=experiment_id):
        # Train model
        model = Ridge(alpha=alpha)
        model.fit(X_train, y_train)

        # Predict and calculate MSE
        y_pred = model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)

        # Log hyperparameters and metrics
        mlflow.log_param("alpha", alpha)
        mlflow.log_metric("mse", mse)

        return {"loss": mse, "status": STATUS_OK}

# Define search space
search_space = {
    "alpha": hp.loguniform("alpha", np.log(0.0001), np.log(10))
}


### Log Experiment Runs

In this section, we'll create all the logged entities that we'll want in scope for migration, including:
 - Model Artifacts
    - Model input_example
 - Model Tags
 - Model Metrics
 - Model Parameters

**NOTE**: Currently, evaluations, and Evaluation Datasets are not in scope due to complexity of changes to the datasets class and the ability to have evaluations runs separate from model logging runs. The legacy mlflow framework doesn't have a reliable way to marry which evaluation run goes with which model and further a consistant way to recover the evaluation dataset used in each model run. This ambiguiity is addressed in MLFlow 3 preview which now has `logged_model` as a first class entity and evaluation runs run against `logged_models` not an arbitrary experiment run.

**NOTE**: As lower priority, will look to migrate the experiment runs and logged models that exist within the same expereiment_run as there is high confidence that the evaluation run is associated with the logged model in the experiment run.

----

The Model Logging process below, will simply keep creating experiement runs until there are four. These runs should be identical and only vary by the experiment run_id and name.

**NOTE**: This does create exceptions in the expereiment run that we want. Specifically, that the `model_signature` isn't populated. We will want this missing information in our replication because we intend to automate the creation of signatures so that model can be future compliant and deployable to `ModelServingV2`.

In [0]:
def exp_runs():
    return client.search_runs(experiment_ids=[experiment_id])

while len(exp_runs()) < 12:
    with mlflow.start_run(experiment_id=experiment_id) as run:
        parent_run_id = run.info.run_id
        best_params = fmin(
            fn=objective,
            space=search_space,
            algo=tpe.suggest,
            max_evals=3,  # Number of trials
            trials=Trials())
        mlflow.set_tags({f"exp_run_tag_{i}": i for i in range(3)})
        mlflow.log_metrics({f"exp_run_metric_{i}": i for i in range(3)})
        mlflow.log_params(best_params)


# Register the best Model of each experiment

A typical process is to register the best model based upon best metrics. That's what we'll do here, we'll register only one of each of the parent models.

In [0]:
df = spark.read.format("mlflow-experiment").load("2743213759305219")
display(df)

In [0]:
experiment_id

In [0]:
%sql

SELECT run_id, start_time, status 
FROM mlflow.runs 
WHERE experiment_id = '2743213759305219' 
AND tags.mlflow.parentRunId IS NULL;

In [0]:
return client.search_runs(experiment_ids=[experiment_id],
                              filter_string="tags.mlflow.rootRunId='507af0bc3ffa4501839ae9d5045685df'")

In [0]:
def exp_parent_runs():
    return client.search_runs(experiment_ids=[experiment_id],
                              filter_string="tags.mlflow.parentRunId >= ''")

exp_runs = exp_parent_runs()

In [0]:
len(exp_runs)

In [0]:
  'mlflow.parentRunId': 'c5d10d941ed640f38a5fcbb843c0eea7',
  'mlflow.rootRunId': 'c5d10d941ed640f38a5fcbb843c0eea7',

In [0]:
[r for r in exp_runs]

In [0]:
rmvs = client.search_model_versions(filter_string=f'name="{model_name}"')

In [0]:
MlflowClient.get_run(run_id)

In [0]:
dir(client.search_registered_models)


# --->  WIP   <---

In [0]:
run_ids = [(r.info.run_id, r.info.artifact_uri) for r in exp_runs() if r.data._tags.get('mlflow.parentRunId', None) is not None]
run_id, artifact_uri = run_ids[0]

In [0]:
run_id='d6a8f65be11b47889112ac142230c2ac'

In [0]:
dir(client)

In [0]:
run = mlflow.get_run(run_id=run_id)

In [0]:
exp = mlflow.get_experiment(experiment_id=experiment_id)

In [0]:
mv = client.get_model_version(name="nested", version="2")

In [0]:
mv

In [0]:
?client.search_model_versions

In [0]:
dir(client)

In [0]:
mv

In [0]:
registered_model_version_meta = {"name": "nested",
                                   "source": f"{artifact_uri}/{model_name}",
                                       "run_id": run_id}
client.create_model_version(**registered_model_version_meta)

In [0]:
def exp_runs():
    return client.search_runs(experiment_ids=[experiment_id], filter_string=f"run_id='d6a8f65be11b47889112ac142230c2ac'")

run = exp_runs()[0]

In [0]:
run.info

In [0]:
client.search_model_versions()

In [0]:
?client.search_runs


# Create workspace Registered Model

When using the legacy `databricks` registry, there isn't support to create the registered model artifact at the same time as registering a model version, thus, we will need to creaste the registered model as a separate task. Further, to make sure that we have the registered model tags migrated, we will also create those tags here.

In [0]:
def rm_tags():
    return client.search_registered_models(experiment_ids=[experiment_id])[0].tags

mlflow.set_registry_uri("databricks")
registered_model_name = model_name
try:
    client.create_registered_model(name=registered_model_name)
    for i in range(3):
        client.set_registered_model_tag(registered_model_name, f"rm_tag_{i}", i)
except:
    registered_model_name = client.search_registered_models(filter_string=f'name="{registered_model_name}"')[0].name


### Register experiment runs as WS registered model verions

We will now register each of the best experiment runs (which is most common practice).

REF: https://mlflow.org/docs/latest/model-registry.html?highlight=stage#migrating-from-stages

In [0]:
def rm_versions():
    return client.search_model_versions(filter_string=f"registered_model_name='{registered_model_name}'")

if len(rm_versions())==0:
    registered_model_versions_meta = [{"name": "orphaned",
                                       "source": f"{r.info.artifact_uri}/{model_name}",
                                       "run_id": r.info.run_id,
                                       "tags": {f"rm_version_tag_{i}": i for i in range(3)}} for i, r in enumerate(exp_runs())]
    for rmv in registered_model_versions_meta:
        client.create_model_version(**rmv)
    for rmv in rm_versions():
        if rmv.version == '3':
            client.transition_model_version_stage(name=registered_model_name, version=3, stage="Production")
        elif rmv.version == '4':
            client.transition_model_version_stage(name=registered_model_name, version=4, stage="Staging")
        elif rmv.__dict__.get('current_stage', 'None') != 'None':
            client.transition_model_version_stage(name=registered_model_name, version=4, stage="None")


## Orphan Registered Models 1 & 3

To replicate the issues of missing parent models which is a problem in migrations, we'll now delete the experiment runs cooresponding to those registered model versions.

In [0]:
orphaned_run_ids = set([rmv.run_id for rmv in rm_versions() if rmv.version in ['1','3']])
experiment_run_ids = set([r.info.run_id for r in exp_runs()])

for run_id in orphaned_run_ids.intersection(experiment_run_ids):
    client.delete_run(run_id)


## Deleted `Orphaned` Replications (Experiment + Registered_Model)

To be able to easily reset the environment while testing, the following code can be used to clean up replication examples.

In [0]:
DELETE_REPLICATION = False

if DELETE_REPLICATION:
    registered_model_list = client.search_registered_models(filter_string=f'name="{registered_model_name}"')    
    if len(registered_model_list) > 0:
        for v in [int(rmv.version) for rmv in rm_versions() if rmv.current_stage!='None']:
            client.transition_model_version_stage(name=registered_model_name, version=v, stage="None")
        client.delete_registered_model(name=registered_model_name)
    experiment_list = client.search_experiments(filter_string=f'name="{experiment_name[10:]}"')
    if len(experiment_list) > 0:
        client.delete_experiment(experiment_id)